In [1]:
score = {1: 2+4+4, 2: 3+2, 3: 2+2+5+5+3, 4: 6+4+4, 5: 2+2}
sum(score.values())

50

In [2]:
import sys
# path to the folder with module
path = '/home/jupyter-slava/teaching/lib/bsm'
sys.path.append(path)

In [3]:
from bsm1 import BSM, Binomial
from numpy import exp

# Exam

* __When you calculate the price of instruments below, it is important you indicate the currency, in which price is calculated. Small deduction will be taken for not indicating currency.__ 

For example, the price is equal to 5.65 USD, or P = 2.3 GBP. But not price = 5.65. 

### Problem 1: Forward

The current GBP/USD exchange rate is $S = 1.25$. <br/>
The risk free interest rates are $2\%$ (for GBP) and $1\%$ (for USD) (continuous compounding). <br/>

a) __(2 points)__ Determine 9 months forward rate for GBP/USD exchange rate.<br/>

In [4]:
S, r, q, T = 1.25, 0.01, 0.02, 9/12

In [5]:
F9m = exp((r-q)*T)*S
print("F9m= {0:.4f}".format(F9m))

F9m= 1.2407


b)  __(4 points)__ The bank enters the forward contract to sell $100m$ GBP to USD at 9m forward rate (calculated in a)). 

 In 3 months the exchange rate changed to $1.2$ , and USD interest rate goes down to $0\%$. 

- Marked to market the forward contract.<br/>

c) __(4 points)__ Marked to market the forward contract using alternative approach by switching asset and money. <br/>




In [6]:
# b)
N = 100
S1, r1, T1 = 1.2, 0.0, 6/12
Fnew = exp((r1-q)*T1)*S1
# short position
V = -exp(-r1*T1)*(Fnew-F9m)*N
print("Fnew= {0:.4f}".format(Fnew))
print("V= {0:.3f} M USD".format(V))

Fnew= 1.1881
V= 5.260 M USD


In [7]:
# c)
N=100*F9m
# X=1/S
X, r, q, T = 1/S, 0.02, 0.01, 9/12
XF9m = exp((r-q)*T)*X
X1, r1, q1, T1 = 1/S1, r, 0, 6/12
XFnew = exp((r1-q1)*T1)*X1
VGBP = exp(-r1*T1)*(XFnew-XF9m)*N
VUSD = VGBP*S1
print("Fnew= {0:.3f}".format(XFnew))
print("V= {0:.3f} M GBP".format(VGBP))
print("V= {0:.3f} M USD".format(VUSD))

Fnew= 0.842
V= 4.383 M GBP
V= 5.260 M USD


<br>

### Problem 2

The same market data as in Problem 1. a) point:

The current GBP/USD exchange rate is $S = 1.25$. <br/>
The risk free interest rates are $2\%$ (for GBP) and $1\%$ (for USD) (continuous compounding).

a) __(3 points)__  Which option is more expensive, European Call option with strike $K=1.25$ (a right to buy 1 GBP for 1.25 USD) and maturity $T=1$, or European Put option with the same strike and maturity? Do you have enough info to answer this question? If not, which additional info do you need?


b) __(2 points)__ Which option is more expensive, European Put option with strike $K=1.3$  and maturity $T=1$, or European Call option with with strike $K=1.25$ and the same maturity? Do you have enough info to answer this question?




### _Solution_

a)

by P-C parity:

$C(K)-P(K) = V(K) = Se^{-yT} - Ke^{-xT}$



In [8]:
K, T = 1.25, 1
VK = exp(-q*T)*S - exp(-r*T)*K
VK

0.012313950553016095

Since $V(K)<0$, it follows $C<P$

b)

Since P(K=1.3) > P(K=1.25) (Put is an increasing function of K), and from a) 


<br>

### Problem 3

The current GBP/USD exchange rate is $S = 1.25$. <br/>

__The risk free interest rates are $0\%$ (for GBP)__ and $1\%$ (for USD) (continuous compounding). Additionally, volatility $\sigma = 12\%$. <br/>

Pay attention, that GBP interest rate is __zero__ now.


### _Solution_

a) __(2 points)__  Build 4-period binomial tree for the exchange rate for the total time horizon of T=12 months (it mean each period length is $\Delta t$=3 month) (also calculate u,d,p, and probabilities of all nodes)

In [9]:
S, r, T, vol = 1.25, 0.01, 12/12, .12

# create class instance a (Tree object)
a = Binomial(S,T,r,0,vol,4)
a.ShowTreeS()
print("")
a.ShowTreeProb()

Tree for S
1.25   
1.18   1.33   
1.11   1.25   1.41   
1.04   1.18   1.33   1.50   
0.98   1.11   1.25   1.41   1.59   

Tree for Probs
1.00   
0.49   0.51   
0.24   0.50   0.26   
0.12   0.37   0.38   0.13   
0.06   0.24   0.37   0.26   0.07   


b) __(2 points)__ Calculate the price of Europian Call option with strike $K = 1.25$ GBP/USD. Maturity $T = 6m$. 


In [10]:
K = 1.25
C6m = a.df**2*(a.TreeProb[2][2]*(a.TreeS[2][2]-K))
print("Call = {0:.3f} USD".format(C6m))

Call = 0.041 USD


c) __(4 points)__ Calculate the price of Barrier Call with strike $K = 1.25$ GBP/USD with __knock-out__ barrier at level $B= 1.2$. Maturity of the option is $1Y$.


Paths in the money: $uuuu$, $uduu$, $uudu$, $uuud$


In [11]:
# expected payoff related to uuuu
Pf1 = a.p**4*(a.TreeS[4][4]-K)
# expected payoff related to the rest of paths
Pf2 = a.p**3*(1-a.p)*(a.TreeS[4][3]-K)
# Price
BC = a.df**4*(Pf1 + 3*Pf2)
print("Barrier Call = {0:.3f} USD".format(BC))

Barrier Call = 0.052 USD


d) __(5 points)__ How to hedge Barrier option from c) at time $t= 3$ months at node (1,1) (up time from intial level) using underlying asset.

In [20]:
# from node (1,1) we do to up: (2,2) or down: (2,1) 
# consider (2,2); 3 paths are in the money: uu, ud, du
# similar to point c)
Cup = a.df**2*(a.p**2*(a.TreeS[4][4]-K) + 2*a.p*(1-a.p)*(a.TreeS[4][3]-K))
# consider (2,1); 1 path is in the money: uu
Cd = a.df**2*a.p**2*(a.TreeS[4][3]-K)
Delta = (Cup - Cd)/(a.TreeS[2][2]-a.TreeS[2][1])
Delta

0.7845093876887156

__Hedge__: Long Barrier Call and short delta GBP/USD

e) __(3 points)__ Calculate the current price of derivative, which pays 100 GBP at time $T$ if GBP/USD exchange $S_T<1.25$.

In [14]:
a.df**4*100*(a.TreeProb[4][0]*a.TreeS[4][0] + a.TreeProb[4][1]*a.TreeS[4][1])

32.60283588883036

<br>

### Problem 4.1

__Market data for 4.1 and 4.2__: The same market data as in Problem 1. a) point:

The current GBP/USD exchange rate is $S = 1.25$. <br/>
The risk free interest rates are $2\%$ (for GBP) and $1\%$ (for USD) (continuous compounding). Volatility $\sigma = 12\%$. <br/>  <br/>


 __(6 points)__ Consider 6 month DCD (dual/multi currency deposit). The notional is $N =1000$ GBP. In the case, GBP will be greater that $K=1.4$ at maturity, the holder will receive $N*K$ USD instead of notional N GBP and __NO Interest__.. 

So payoff can be written as:


- $e^{(r_{GBP}+y)T}N$ GBP if $S_T < K$

- $NK$ USD if $S_T >= K$



Find the replication portfolio and calculate fair yield enhancement $y$.


### _Solution_

Can be replicated as 

* deposit in GBP paying at $T$: $e^{(r_{GBP}+y)T}N$. It is current value is $e^{yT}NS_0$ USD.

* short Call with notional $e^{(r_{GBP}+y)T}N$;
    
  (payoff is $e^{(r_{GBP}+y)T}NK$ USD if $S_T>K$)
  
* short Binary paying notional $NK (e^{(r_{GBP}+y)T}-1)$ in USD if $S_T>K$


In [37]:
S, r, q = 1.25, 0.01, 0.02
vol = 0.12
wUSD = BSM(S,vol,r,q)

y = 0.0092
T = 6/12
K = 1.4
# N=1
Price = exp(y*T)*S - exp((y+q)*T)*wUSD.Call(K,T) - (1-wUSD.Pr(K,T))*exp(-r*T)*(exp((y+q)*T)-1)*K
print("P0 = {0:.4f} ".format(Price))

P0 = 1.2500 


<br>

### Problem 4.2

__(4 points)__ Calculate the current price of derivative, which pays the min(1.25 mln USD, 1 mln GBP) in 1 year. 

### _Solution_

$min(S,K) = K + min(S-K,0)= K - max(K-S,0)$

Long Discount bond and short Put option

In [38]:
K = S
exp(-r*T)*K - wUSD.Put(K,T)

1.1722853079673419

<br>

### Problem 4.3

__(4 points)__ 

This is an analytical question.

Assume a Black-Scholes world. The underlying stock does not pay dividends and interest rate is zero. Volatility is $\sigma >0$.

Compare prices of two binary options:

1. It pays N USD if $S_T > S_0$

2. It pays N USD if $S_T < S_0$

Do you have enough info to find out, which option is more expensive. Justify your answer.



### _Solution_

The price of digital is equal to $N Pr\{A\}$, where $A$ is an event. $S_T > S_0$ or $S_T < S_0$ in our case.

$S_T = S_0e^{(-\sigma^2/2)T + \sigma \sqrt{T} \epsilon}$, or $S_T = S_0e^{y_T}$, 
where $y_T \sim N(-\sigma^2T/2 ,\sigma^2 T)$.  We took into account that $r=0$.

It follows that $S_T > S_0$ is equavalent to $y_T > 0$. 

Since $E(y_T)<0 \implies \Pr(y_T > 0)<\frac{1}{2}$ and $\Pr(y_T < 0)>\frac{1}{2}$.

It implies than the second binary is more expensive than the first one.

### Problem 5

__a)__ (2 points) Gold mining company plan to enter a forward contract with the bank. According to the contract, the bank will sell 1000 ounces of gold to the mining company at maturity for the price, which is fixed now. Is it right, wrong way risk or no risk at all for the bank, and explain why?

__b)__ (2 points) A bank bought PUT option on USD/RUB exchange rate from the importer? Is it right, wrong way risk or no risk at all for the bank, and explain why? Definition of an importer: revenures in RUB, cost in USD.

### _Solution_

__a)__ wwr: bank profit from contract when gold goes down, not good situation for mining company.

__b)__ rwr: bank profit from contract when USD/RUB goes down, a good situation for  the importer.